In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
calls=pd.read_csv('calls.csv')
customers=pd.read_csv('customer.csv')
reasons= pd.read_csv('reason.csv')
sentiments= pd.read_csv('sentiment.csv')

In [ ]:
 pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
calls['call_start_datetime']=pd.to_datetime(calls['call_start_datetime'])
calls['call_end_datetime']=pd.to_datetime(calls['call_end_datetime'])
calls['agent_assigned_datetime']=pd.to_datetime(calls['agent_assigned_datetime'])
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71810 entries, 0 to 71809
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   call_id                  71810 non-null  int64         
 1   customer_id              71810 non-null  int64         
 2   agent_id                 71810 non-null  int64         
 3   call_start_datetime      71810 non-null  datetime64[ns]
 4   agent_assigned_datetime  71810 non-null  datetime64[ns]
 5   call_end_datetime        71810 non-null  datetime64[ns]
 6   call_transcript          71810 non-null  object        
dtypes: datetime64[ns](3), int64(3), object(1)
memory usage: 3.8+ MB


In [ ]:
merged_df1= calls.merge(reasons, on='call_id', how='left') \
                      .merge(customers, on='customer_id', how='left') \
                      .merge(sentiments, on='call_id', how='left')
merged_df1.head()

,call_id,customer_id,agent_id_x,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,primary_call_reason,customer_name,elite_level_code,agent_id_y,agent_tone,customer_tone,average_sentiment,silence_percent_average
0,4667960400,2033123310,963118,2024-07-31 23:56:00,2024-08-01 00:03:00,2024-08-01 00:34:00,\n\nAgent: Thank you for calling United Airlin...,Voluntary Cancel,Matthew Foster,4.0,963118,neutral,angry,-0.04,0.39
1,1122072124,8186702651,519057,2024-08-01 00:03:00,2024-08-01 00:06:00,2024-08-01 00:18:00,\n\nAgent: Thank you for calling United Airlin...,Booking,Tammy Walters,NaN,519057,calm,neutral,0.02,0.35
2,6834291559,2416856629,158319,2024-07-31 23:59:00,2024-08-01 00:07:00,2024-08-01 00:26:00,\n\nAgent: Thank you for calling United Airlin...,IRROPS,Jeffery Dixon,NaN,158319,neutral,polite,-0.13,0.32
3,2266439882,1154544516,488324,2024-08-01 00:05:00,2024-08-01 00:10:00,2024-08-01 00:17:00,\n\nAgent: Thank you for calling United Airlin...,Upgrade,David Wilkins,2.0,488324,neutral,frustrated,-0.20,0.20
4,1211603231,5214456437,721730,2024-08-01 00:04:00,2024-08-01 00:14:00,2024-08-01 00:23:00,\n\nAgent: Thank you for calling United Airlin...,Seating,Elizabeth Daniels,0.0,721730,neutral,polite,-0.05,0.35


In [ ]:
merged_df1['Handling_time']=(merged_df1['call_end_datetime']-merged_df1['agent_assigned_datetime']).dt.total_seconds()
merged_df1['Speed_to_answer_time']=(merged_df1['agent_assigned_datetime']-merged_df1['call_start_datetime']).dt.total_seconds()
merged_df1['WeekDay_datetime']=merged_df1['call_start_datetime'].dt.dayofweek
merged_df1['Hour_datetime']=merged_df1['call_start_datetime'].dt.hour

merged_df1.head()

,call_id,customer_id,agent_id_x,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,primary_call_reason,customer_name,elite_level_code,agent_id_y,agent_tone,customer_tone,average_sentiment,silence_percent_average,Handling_time,Speed_to_answer_time,WeekDay_datetime,Hour_datetime
0,4667960400,2033123310,963118,2024-07-31 23:56:00,2024-08-01 00:03:00,2024-08-01 00:34:00,\n\nAgent: Thank you for calling United Airlin...,Voluntary Cancel,Matthew Foster,4.0,963118,neutral,angry,-0.04,0.39,1860.0,420.0,2,23
1,1122072124,8186702651,519057,2024-08-01 00:03:00,2024-08-01 00:06:00,2024-08-01 00:18:00,\n\nAgent: Thank you for calling United Airlin...,Booking,Tammy Walters,NaN,519057,calm,neutral,0.02,0.35,720.0,180.0,3,0
2,6834291559,2416856629,158319,2024-07-31 23:59:00,2024-08-01 00:07:00,2024-08-01 00:26:00,\n\nAgent: Thank you for calling United Airlin...,IRROPS,Jeffery Dixon,NaN,158319,neutral,polite,-0.13,0.32,1140.0,480.0,2,23
3,2266439882,1154544516,488324,2024-08-01 00:05:00,2024-08-01 00:10:00,2024-08-01 00:17:00,\n\nAgent: Thank you for calling United Airlin...,Upgrade,David Wilkins,2.0,488324,neutral,frustrated,-0.20,0.20,420.0,300.0,3,0
4,1211603231,5214456437,721730,2024-08-01 00:04:00,2024-08-01 00:14:00,2024-08-01 00:23:00,\n\nAgent: Thank you for calling United Airlin...,Seating,Elizabeth Daniels,0.0,721730,neutral,polite,-0.05,0.35,540.0,600.0,3,0


In [ ]:
merged_df1['elite_level_code']=merged_df1['elite_level_code'].fillna(0)
#merged_df1.dropna(inplace=True)
merged_df1.isnull().sum()



,0
call_id,0
customer_id,0
agent_id_x,0
call_start_datetime,0
agent_assigned_datetime,0
call_end_datetime,0
call_transcript,0
primary_call_reason,5157
customer_name,0
elite_level_code,0


In [ ]:
# Normalize spacing in the primary_call_reason column
merged_df1['primary_call_reason'] = (
        merged_df1['primary_call_reason']
        .str.strip()  # Remove leading/trailing whitespace
        .str.replace(r'\s+', ' ', regex=True))
merged_df1['primary_call_reason'] = (
        merged_df1['primary_call_reason'].str.replace('-', ' ', regex=True))
merged_df1['primary_call_reason'] =(
        merged_df1['primary_call_reason'] .str.replace('&', 'and',regex=True))
merged_df1.primary_call_reason.unique()

array(['Voluntary Cancel', 'Booking', 'IRROPS', 'Upgrade', 'Seating',
       'Mileage Plus', 'Checkout', nan, 'Voluntary Change', 'Post Flight',
       'Check In', 'Other Topics', 'Communications', 'Schedule Change',
       'Products and Services', 'Digital Support', 'Disability',
       'Unaccompanied Minor', 'Baggage', 'Traveler Updates', 'ETC'],
      dtype=object)

In [ ]:
print(pd.DataFrame(merged_df1['call_transcript'].apply(len))['call_transcript'].mean())


2781.7814371257487


In [ ]:
from openai import OpenAI
import os
OpenAI.api_key = 'sk-Gc0dkfTK4tOhnhX5aGR0ZIsA-zBS52Jbo_PK_sQAeCT3BlbkFJYssPylg8y-dPFlwvbcKEDZTza_VxUkkhStSwta7iMA' # update your API key here
os.environ['OPENAI_API_KEY'] = 'sk-Gc0dkfTK4tOhnhX5aGR0ZIsA-zBS52Jbo_PK_sQAeCT3BlbkFJYssPylg8y-dPFlwvbcKEDZTza_VxUkkhStSwta7iMA'# update your API key here
client = OpenAI()

def create_reason(transcript):

  completion = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": """You are an airlines industry expert working for a global airlines which caters to both domestic and global customers.
             You are in charge of improving the efficiency and optimising the operations in a call centre which which handles customer inquiries, complaints, and service requests.
             your first objective is to understand the call transcripts and accurately categorise primary reasons for the customers calls.

             As a first priority you have to try and categorise the primary reason into the following categories

             Baggage ( includes topics like Baggage fee, Carry on-bags, checked bags, damaged bags,  delayed bags, claim status, lost items, bag policies and other similar topics ) ,
             Booking ( includes topics like Booking a flight, flexible booking options, Travel Insurance, sustainable travel,fare lock, reserve trip, pay later and other similar topics ) ,
             Check In ( includes topics like Check-in options, Check-in time limits, security, Boarding, airport reaching time and other similar topics ),

             Disability ( includes topics like Wheelchair assistance, accessible travel, accessible seating, service dogs, assistance, special needs and other similar topics ) ,
             ETC ( includes topics future flight credits, travel credits, travel certificates, travel credits FAQ and other similar topics ),
             IRROPS ( includes topics like irregular operations, weather conditions, travel restrictions, diversions, re-routed flights, flight delays and other similar topics ),
             Mileage Plus ( includes topics like Mileage plus number, Travel bank, change name in account, locked account, join mileage plus, buy miles, transfer miles, missing miles, mileage plus service center, Mileage plus premier and other similar topics ) ,
             Post Flight ( includes topics which give feedback on flight service, timings, quality, comfort, user experience and other similar topics discussing feedback/complaints from the customer after flight is completed ),
             Products and Services ( includes topics like onboard services, food and drinks, inflight dining and other similar topics ) ,
             Schedule Change ( includes topics like flight schedule change and other similar topics ) ,
             Seating ( includes topics like seat options, change seats, basic economy, economy plus, booking inquiries specific to seat no, booking inquiries specific to seating position),
             Traveler Updates ( includes topics like Travel requirements, guidelines, travel alerts and similar topics ) ,
             Unaccompanied Minor ( includes topics like Unaccompanied minors, Minors travelling alone, Kids between ages 5 to 14 years travelling alone, optional for kids between 15 to 17 years travelling alone) ,
             Upgrade ( includes topics like Upgrades, upgrades with miles, upgrade awards -in options, Check-in time limits, security, Boarding and other similar topics ) ,
             Voluntary Cancel ( includes topics like voluntary cancellation of flight and other similar topics ) ,
             Voluntary Change ( includes topics like voluntary change of flight and other similar topics )
             Other Topics ( if you can not categorise the primary reason for call into one of the above categories create a new category which gives the primary reason for the call in not more than 3 words ),

             Always provide your final answer as one of the above categories only, no other text is required in your final answer """},
            {
                "role": "user",
                "content": f"""
                      {transcript}
                """
            }
        ]
    )


#transcripts for these issues mostly had conversations about Schedule change, voluntary change, seating and booking instead of these categories and thus it provides an opportunity to simplify our future tagging process. If need be these can be later on ,
# Check out
# Communications,
# Digital Support,


  return completion.choices[0].message.content


In [ ]:
filtered_df = merged_df1[merged_df1['primary_call_reason'].notnull()]
sampled_Non_null_df = filtered_df.sample(2500, random_state=5)
sampled_Non_null_df.shape

# sum(sampled_Non_null_df['Created_Primary reason']==sampled_Non_null_df['primary_call_reason'])
sampled_Non_null_df['Created_Primary reason']=sampled_Non_null_df['call_transcript'].apply(create_reason)


# sampled_Non_null_df.to_excel('sampled_df.xlsx', index=False)

merged_df2=merged_df1[merged_df1['primary_call_reason'].isnull()].copy()
merged_df2.shape

merged_df2['Created_Primary reason']=merged_df2['call_transcript'].apply(create_reason)


In [ ]:
sampled_Non_null_df[['call_id','call_start_datetime','call_transcript','primary_call_reason','Created_Primary reason']].head(500)
sampled_Non_null_df.head(500).to_excel('sampled_Non_null_df_first_500.xlsx', index=False)

,call_id,call_start_datetime,call_transcript,primary_call_reason,Created_Primary reason
52065,7910660028,2024-08-24 09:09:00,\n\nAgent: Thank you for calling United Airlin...,Communications,Voluntary Change
62368,753104781,2024-08-27 11:46:00,\n\nAgent: Thank you for calling United Airlin...,IRROPS,Voluntary Change
57442,5074213514,2024-08-25 10:51:00,\n\nAgent: Thank you for calling United Airlin...,IRROPS,IRROPS
33275,7620009390,2024-08-15 16:40:00,\n\nAgent: Thank you for calling United Airlin...,Products and Services,IRROPS
46120,6815361592,2024-08-20 08:28:00,\n\nAgent: Thank you for calling United Airlin...,Other Topics,Voluntary Change
...,...,...,...,...,...
33827,198229427,2024-08-16 08:41:00,\n\nAgent: Thank you for calling United Airlin...,Booking,Voluntary Change
20374,959118947,2024-08-10 10:34:00,\n\nAgent: Thank you for calling United Airlin...,Mileage Plus,Voluntary Change
45100,301559938,2024-08-19 12:07:00,\n\nAgent: Thank you for calling United Airlin...,IRROPS,IRROPS
56479,9686323204,2024-08-25 08:13:00,\n\nAgent: Thank you for calling United Airlin...,Schedule Change,Voluntary Change


In [ ]:
merged_df2[['call_id','call_start_datetime','call_transcript','primary_call_reason','Created_Primary reason']].head(500)
merged_df2.head(500).to_excel('sampled_null_df_first_500.xlsx', index=False)

,call_id,call_start_datetime,call_transcript,primary_call_reason,Created_Primary reason
11,7732610078,2024-08-01 00:23:00,\n\nAgent: Thank you for calling United Airlin...,NaN,Voluntary Change
33,2400299738,2024-08-01 01:33:00,\n\nAgent: Thank you for calling United Airlin...,NaN,Post Flight
48,6533095063,2024-08-01 02:17:00,\n\nAgent: Thank you for calling United Airlin...,NaN,Voluntary Change
56,7774450920,2024-08-01 02:49:00,\n\nAgent: Thank you for calling United Airlin...,NaN,Post Flight
57,9214147168,2024-08-01 02:49:00,\n\nAgent: Thank you for calling United Airlin...,NaN,Schedule Change
...,...,...,...,...,...
6867,8855925714,2024-08-03 18:23:00,\n\nAgent: Thank you for calling United Airlin...,NaN,Voluntary Change
6891,7323677077,2024-08-03 18:33:00,\n\nAgent: Thank you for calling United Airlin...,NaN,IRROPS
6900,9368418712,2024-08-03 18:32:00,\n\nAgent: Thank you for calling United Airlin...,NaN,Voluntary Change
6927,3831498185,2024-08-03 18:38:00,\n\nAgent: Thank you for calling United Airlin...,NaN,Schedule Change
